In [ ]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import copy
import numpy as np
import math
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import itertools
import pandas as pd

from pycmqlib3.utility import dbaccess, dataseries, misc
import pycmqlib3.analytics.data_handler as dh


import sys
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

sys.path.append("C:/dev/pyktrader3/")
sys.path.append("C:/dev/wtpy/")
sys.path.append("C:/dev/akshare/")
sys.path.append("C:/dev/wtdev/")

from pycmqlib3.analytics.tstool import *
from pycmqlib3.analytics.btmetrics import *
from pycmqlib3.analytics.backtest_utils import *
from misc_scripts.fun_factor_update import *
from pycmqlib3.strategy.signal_repo import *
from bktest.backtest_grid_search import *
from misc_scripts.update_fut_prices import *

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
mpl.rcParams['figure.figsize'] = (12, 8)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
lead_lag_config = {
    'll_left': -20,
    'll_right': 120,
    'll_spacing': 5,
    'll_sub_win': [(datetime.date(2008, 1, 1), datetime.date(2016, 12, 31)), 
                   (datetime.date(2017, 1, 1), datetime.date(2022, 12, 31)),],
}

ll_keys = ['fullsample'] + ['%s:%s' % (sd.strftime('%Y-%b-%d'), ed.strftime('%Y-%b-%d')) for sd, ed in lead_lag_config['ll_sub_win']]


In [ ]:
now = datetime.datetime.now()
tday = now.date()
if (not is_workday(tday, 'CHN')) or (now.time() < datetime.time(17, 0, 0)):
    tday = day_shift(tday, '-1b', CHN_Holidays)
_ = refresh_saved_fut_prices(run_date=tday)

In [ ]:
#tday=pd.Timestamp("20250530")

# product group and starting date

In [ ]:
from misc_scripts.update_fut_prices import load_saved_fut

df = load_saved_fut(tday, freq='d')
spot_df = load_fun_data(tday)

start_date = df.index[0]
end_date = tday

cdates = pd.date_range(start=start_date, end=tday, freq='D')
bdates = pd.bdate_range(start=start_date, end=end_date, freq='C', holidays=misc.CHN_Holidays)


In [ ]:
bdates = [d for d in bdates if d.date() not in [datetime.date(2014,1,2), datetime.date(2014,1,3)]]
df = df.reindex(index=bdates)

In [ ]:
spd_list = [df]
spd_cont_list = []
for spd_name in spread_config:
    asset_list = spread_config[spd_name][0]
    roll_rule=spread_config[spd_name][2]
    nb_cont=spread_config[spd_name][3]
    if (asset_list[0][0], nb_cont) not in spd_cont_list:
        leg1_df = nearby(asset_list[0][0], n=nb_cont,
                         start_date=start_date.date(),
                         end_date=end_date.date(),
                         roll_rule=roll_rule, freq='d', shift_mode=1)
        leg1_df.index = pd.to_datetime(leg1_df.index)
        leg1_df.columns = pd.MultiIndex.from_tuples([(f'{asset_list[0][0]}d{nb_cont}', col) for col in leg1_df.columns])
        spd_list.append(leg1_df)
        spd_cont_list.append((asset_list[0][0], nb_cont))
    if (asset_list[1][0], nb_cont) not in spd_cont_list:
        leg2_df = nearby(asset_list[1][0], n=nb_cont,
                         start_date=start_date.date(),
                         end_date=end_date.date(),
                         roll_rule=roll_rule, freq='d', shift_mode=1)
        leg2_df.index = pd.to_datetime(leg2_df.index)
        leg2_df.columns = pd.MultiIndex.from_tuples([(f'{asset_list[1][0]}d{nb_cont}', col) for col in leg2_df.columns])
        spd_list.append(leg2_df)
        spd_cont_list.append((asset_list[1][0], nb_cont))
df = pd.concat(spd_list, axis=1)

In [ ]:
for asset_cont in df.columns.get_level_values(0).unique():
    if (asset_cont, "d_twap") not in df.columns:
         df[(asset_cont, "d_twap")] = np.nan
    flag = df[(asset_cont, "d_twap")].isna()
    df.loc[flag, (asset_cont, 'd_twap')]= df.loc[flag, (asset_cont, 'close')] 
    if (asset_cont, "a1535") not in df.columns:
         df[(asset_cont, "a1535")] = np.nan    
    flag = df[(asset_cont, "a1535")].isna()
    df.loc[flag, (asset_cont, 'a1535')]= df.loc[flag, (asset_cont, 'd_twap')]                
    if (asset_cont, "a1505") not in df.columns:
         df[(asset_cont, "a1505")] = np.nan    
    flag = df[(asset_cont, "a1505")].isna()
    df.loc[flag, (asset_cont, 'a1505')]= df.loc[flag, (asset_cont, 'a1535')]            
    for traded_price in ['n305', 'n310', 'n315', 'n450']:
        if (asset_cont, traded_price) not in df.columns:
            df[(asset_cont, traded_price)] = np.nan 
        flag = df[(asset_cont, traded_price)].isna()
        df.loc[flag, (asset_cont, traded_price)]= df.loc[flag, (asset_cont, 'a1505')]
        

In [ ]:
asset_pairs = [
    ('rb', 'i'), ('i', 'rb'), ('hc', 'i'), ('j', 'i'), ('jm', 'i'), ('hc', 'zn'), ('SF', 'SM'), 
    ('FG', 'SA'), ('SA', 'FG'), ('ao', 'al'),  ('pp', 'l'), 
    ('m', 'y'), ('y', 'm'), ('m', 'a'), ('y', 'p'), ('p', 'y'), ('OI', 'p'), ('p', 'OI'),
    ('au', 'ag'), ('au', 'cu'), ('au', 'rb'), 
]
beta_ret_dict = {}
beta_dict = {}
beta_win = 244
vwin = 20
for trade_asset, index_asset in asset_pairs:    
    asset_df = df[[(f'{index_asset}c1', 'close'), (f'{trade_asset}c1', 'close')]].copy(deep=True).droplevel([1], axis=1)
    asset_df = asset_df.pct_change().dropna()
    asset_df.columns = [col[:-2] for col in asset_df.columns]    
    for asset in asset_df:
        asset_df[f'{asset}_pct'] = asset_df[asset].rolling(5).mean()
        asset_df[f'{asset}_vol'] = asset_df[asset].rolling(vwin).std()
    asset_df['beta'] = asset_df[f'{index_asset}_pct'].rolling(beta_win).cov(asset_df[f'{trade_asset}_pct'])/asset_df[f'{index_asset}_pct'].rolling(beta_win).var()
    key = '_'.join([trade_asset, index_asset])
    asset_df['pct_chg'] = asset_df[trade_asset] - asset_df['beta'] * asset_df[index_asset].fillna(0)
    asset_df['pct_vol'] = asset_df['pct_chg'].rolling(vwin).std()
    asset_df['trade_leg'] = asset_df[f'{trade_asset}_vol']/asset_df['pct_vol']
    asset_df['index_leg'] = - asset_df[f'{index_asset}_vol'] / asset_df['pct_vol'] * asset_df['beta']
    
    beta_ret_dict[key] = asset_df['pct_chg']
    beta_dict[key] = asset_df[['beta', 'trade_leg', 'index_leg']]
    

In [ ]:
vol_win = 20
product_list = list(set([col[:-2] for col in df.columns.get_level_values(0).unique()]))
spot_dict = {}
spot_df['io_ctd_spot'] = io_ctd_basis(spot_df, df[('ic1', 'expiry')])

for asset in product_list:    
    spot_dict[f'{asset}_px'] = df[(asset+'c1', 'close')]
    spot_dict[f'{asset}_logret'] = np.log(df[(asset+'c1', 'close')]).dropna().diff()
    spot_dict[f'{asset}_pctchg'] = np.log(df[(asset+'c1', 'close')]).dropna().pct_change()        
    if (asset+'c2', 'close') in df.columns:
        spot_dict[f'{asset}_ryield'] = (np.log(df[(asset+'c1', 'close')]) - np.log(df[(asset+'c2', 'close')]) - 
                                      df[(asset+'c1', 'shift')] + df[(asset+'c2', 'shift')])/(df[(asset+'c2', 'expiry')] - df[(asset+'c1', 'expiry')]).dt.days*365.0 + spot_df['r007_cn'].reindex(index=df.index).ffill().ewm(5).mean()/100
        spot_dict[f'{asset}_logret2'] = np.log(df[(asset+'c2', 'close')]).dropna().diff()
        spot_dict[f'{asset}_basmom'] =spot_dict[f'{asset}_logret'] - spot_dict[f'{asset}_logret2']        
        spot_dict[f'{asset}_pctvol'] = spot_dict[f'{asset}_pctchg'].dropna().rolling(vol_win).std()
        spot_dict[f'{asset}_basmom5'] = spot_dict[f'{asset}_basmom'].dropna().rolling(5).sum()
        spot_dict[f'{asset}_basmom10'] = spot_dict[f'{asset}_basmom'].dropna().rolling(10).sum()
        spot_dict[f'{asset}_basmom20'] = spot_dict[f'{asset}_basmom'].dropna().rolling(20).sum()
        spot_dict[f'{asset}_basmom60'] = spot_dict[f'{asset}_basmom'].dropna().rolling(60).sum()
        spot_dict[f'{asset}_basmom120'] = spot_dict[f'{asset}_basmom'].dropna().rolling(120).sum()
        spot_dict[f'{asset}_basmom180'] = spot_dict[f'{asset}_basmom'].dropna().rolling(180).sum()
    if asset in commod_phycarry_dict:
        asset_feature = commod_phycarry_dict[asset]
        tmp_df = pd.concat([spot_df[[asset_feature, 'r007_cn']].dropna(how='all'), 
                            (df[(asset+'c1', 'close')] / np.exp(df[(asset+'c1', 'shift')])).dropna().to_frame(f'{asset}_c1'),
                            pd.to_datetime(df[(asset+'c1', 'expiry')]).dropna().to_frame(f'{asset}_expiry')], axis=1).ffill()
        tmp_df['date'] = pd.to_datetime(tmp_df.index)            
        tmp_df['r007_cn'] = tmp_df['r007_cn'].ewm(5).mean()/100
        spot_dict[f'{asset}_phycarry'] = (np.log(tmp_df[asset_feature]) - np.log(tmp_df[f'{asset}_c1'])) / \
                                            (tmp_df[f'{asset}_expiry'] - tmp_df['date']).dt.days * 365 + tmp_df['r007_cn']    

spot_dict['hc_rb_diff'] = np.log(df[('hcc1', 'close')]) - np.log(df[('rbc1', 'close')])
spot_dict['rb_hc_basmom_diff'] = spot_dict['rb_basmom'] - spot_dict['hc_basmom']
spot_dict['rb_hc_phycarry_diff'] = spot_dict['rb_phycarry'] - spot_dict['hc_phycarry']
 
spot_df = pd.concat([spot_df, pd.DataFrame(spot_dict)], axis=1)

In [ ]:
from pycmqlib3.strategy.signal_repo import *
from misc_scripts.fun_factor_update import *


In [ ]:
broad_mkts = [
    'rb', 'hc', 'i', 'j', 'jm', 'FG', 'v', 'SM', 'SF', 'SA',
    'cu', 'al', 'zn', 'ni', 'pb', 'sn', 'ss', 'si', 'ao', 'au', 'ag',#'bc'
    'ru', 'l', 'pp', 'TA', 'MA', 'sc', 'eb', 'eg', 'UR', 'lu', #'bu', 'fu',
    'm', 'RM', 'y', 'p', 'OI', 'a', 'c', 'CF', 'jd', 'AP', 'lh', #'cs',
    'si', 'lc', 'PX', # 'PK', 'CJ', 'ec', 
]

ind_mkts = ['rb', 'hc', 'i', 'j', 'jm', 'FG', 'v', 'SA', 'UR', 'SM', 'SF', 
    'cu', 'al', 'zn', 'ni', 'pb', 'sn', 'ss', 'ao', 'au', 'ag',#'bc', 'si', 
    'ru', 'l', 'pp', 'TA', 'MA', 'sc', 'eb', 'eg', 'lu', #'bu', 'fu',
]

signal_by_func = {
    **factors_by_func,
    **{ 
    }
}

signal_by_spread = {
    **factors_by_spread,
    **{
        
    }
}

signal_by_spread2 = {
    **factors_by_spread2,
    **{
        
    }
}

signal_by_beta_neutral = {
    **factors_by_beta_neutral,
    **{
        
    }
}

signal_by_asset = {
    **factors_by_asset,
    **{
        "ryield_ema": broad_mkts,
        "ryield_st_zsa": broad_mkts,
        "ryield_lt_zsa": broad_mkts,
        "basmom20_ema": broad_mkts,
        "basmom60_ema": broad_mkts,
        "basmom120_ema": broad_mkts,
        "mom_hlr_st": broad_mkts,
        "mom_hlr_lt": broad_mkts,
        "mom_momma20": broad_mkts,
        "mom_momma240": broad_mkts,
        'metal_pbc_ema': ['cu', 'al', 'zn', 'ni', 'ss', 'sn', 'ao', 'au', 'ag',
                          'rb', 'hc', 'i', 'j', 'jm', 'v', 'FG', 'SA', ],
        'metal_mom_hlrhys': ['cu', 'al', 'zn', 'pb', 'ni', 'ss', 'sn', 'ao', 'au', 'ag',
                             'rb', 'hc', 'i', 'j', 'jm', 'SM', 'SF', 'v', 'FG', 'SA'],
        'metal_inv_hlr': ['cu', 'al', 'zn', 'pb', 'ni', 'ss', 'sn', 'ao',
                          'rb', 'hc', 'i', 'v', 'FG', 'SA'],
        'metal_inv_lyoy_hlr': ['cu', 'al', 'zn', 'pb', 'ni', 'ss', 'sn', 'ao', 
                               'rb', 'hc', 'i', 'v', 'FG', 'SA'], 
        
        'lme_base_ts_mds': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'lme_base_ts_hlr': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'lme_futbasis_ma': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'base_inv_shfe_ma': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'base_inv_lme_ma': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'base_inv_exch_ma': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'base_phybas_carry_ma': ['cu', 'al', 'zn', 'ni', 'sn', 'pb'],
        'base_inv_mds': ['cu', 'al', 'zn', 'ni', 'sn', 'pb', 'ss', 'si', 'ao'],
        'base_tc_1y_zs': ['cu', 'pb', 'zn'],
        'base_tc_2y_zs': ['cu', 'pb', 'sn'],
        'base_cifprem_1y_zs': ['cu', 'al', 'zn', 'ni'],
        'base_phybasmom_1m_zs': ['cu', 'al', 'zn', 'ni', 'pb', 'sn'],
        'base_phybasmom_1y_zs': ['cu', 'al', 'zn', 'ni', 'pb', 'sn'],
    }
}

signal_by_data = {
    **single_factors,
    **{
        'steel_margin_lvl_fast': ['i', 'j', 'jm', 'SM', 'SF'],
        'steel_margin_lvl_slow': ['SM', 'SF'],
        'strip_hsec_lvl_mid': ['rb', 'hc', 'i', 'j', 'jm'],
        'io_removal_lvl': ['i'],
        'io_removal_lyoy': ['i'],
        'io_millinv_lyoy': ['hc', 'i'],
        'io_inv_rmv_ratio_1y': ['i'],
        'ioarb_px_hlr': ['rb', 'hc', 'i'],
        'ioarb_px_hlrhys': ['rb', 'hc', 'i'],
        'steel_sinv_lyoy_zs': ['rb', 'hc', 'i', 'FG', 'v'],
        'steel_sinv_lyoy_mds': ['rb', 'hc', 'i', 'FG', 'v'],
        'rbsales_lyoy_mom_lt': ['rb'],
        'rb_sales_inv_ratio_lyoy': ['rb'],
        'fef_c1_c2_ratio_or_qtl': ['rb', 'hc', 'j'],
        'fef_fly_ratio_or_qtl': ['rb', 'hc', 'j'],
        'fef_basmom_or_qtl': ['rb', 'hc'],
        'fef_basmom5_or_qtl': ['rb', 'hc'],      
        "const_etf_mom_zsa": ["rb", "i", "j", "FG", "v"],
        "const_etf_mom_ewm": ["rb", "i", "j", "FG", "v"],
        "prop_etf_mom_dbth_zs": ["rb", "i", "FG", "v"],
        "prop_etf_mom_dbth_qtl": ["rb", "i", "FG", "v"],
        "prop_etf_mom_dbth_qtl2": ["rb", "i", "FG", "v"],  
        
        "al_alumina_qtl": ['al'],
        "al_alumina_yoy_qtl": ['al'],
        "al_coal_qtl": ['al'],
        "ni_nis_mom_qtl": ['ni'],
        "ni_ore_qtl": ['ni'],
        "sn_conc_spot_hlr": ['sn'],
        'cu_prem_usd_zsa': ['cu'],
        'cu_prem_usd_md': ['cu'],
        'cu_phybasis_zsa': ['cu'],
        'cu_phybasis_hlr': ['cu'],
        "base_etf_mom_zsa": ["cu", "al", "zn", "pb", "ni", "sn"],
        "base_etf_mom_ewm": ["cu", "al", "zn", "pb", "ni", "sn"],        
    }
}

feature_setup = {
    **signal_store,
    **{      
    }
}

# strategy portfolio config

In [ ]:
prem_fact = 25000
metal_fact = 12000
misc_fact = 14000
ferr_fact = 25000
base_fact = 25000
energy_fact = 3000
macro_fact = 10000
macro2_fact = 3000
seazn_fact = 12000
bond_fact = 50000
ferspd_fact= 70000
auspd_fact= 30000

prem_strats = [
    ["ryield_ema", 0.082*prem_fact], 
    ["ryield_ema_xdemean", 0.20*prem_fact], 
    ["ryield_st_zsa", 0.016*prem_fact],
    ["ryield_st_zsa_xdemean", 0.02*prem_fact],
    ["ryield_lt_zsa", 0.025*prem_fact],
    ["ryield_lt_zsa_xdemean", 0.045*prem_fact],
    
    ["basmom20_ema", 0.013*prem_fact],
    ["basmom20_ema_xdemean", 0.032*prem_fact],           
    ["basmom60_ema", 0.022*prem_fact],
    ["basmom60_ema_xdemean", 0.027*prem_fact], 
    ["basmom120_ema", 0.02*prem_fact],
    ["basmom120_ema_xdemean", 0.035*prem_fact],

    ["mom_momma240", 0.02*prem_fact],
    ["mom_momma240_xdemean", 0.062*prem_fact],
    ["mom_momma20", 0.009*prem_fact],
    ["mom_momma20_xdemean", 0.0105*prem_fact],     
    ["mom_hlr_st", 0.062*prem_fact],
    ["mom_hlr_st_xdemean", 0.04*prem_fact], 
    ["mom_hlr_lt_xdemean", 0.014*prem_fact],
    ["mom_hlr_yr_xdemean", 0.00*prem_fact],   
]

metal_strats = [
#     ['metal_pbc_ema', 0.457*metal_fact],
#     ['metal_pbc_ema_xdemean', 0.315*metal_fact],    
    ['metal_pbc_ema', 0.316*metal_fact],
    ['metal_pbc_ema_xdemean', 0.632*metal_fact],        
    ['metal_mom_hlrhys', 0.316*metal_fact],
    ['metal_mom_hlrhys_xdemean', 0.042*metal_fact],    
    ['metal_inv_hlr', 0.093*metal_fact],
    ['metal_inv_hlr_xdemean', 2.26*metal_fact],
#     ['metal_inv_lyoy_hlr', 0],        
]

misc_strats = [    
    ["exch_wnt_hlr", 0.48*misc_fact],
    ["exch_wnt_hlr_xdemean", 0.36*misc_fact],
    ["exch_wnt_yoy_hlr", 0.12*misc_fact],
    ["exch_wnt_yoy_hlr_xdemean", 0.24*misc_fact],        
    ["leadlag_d_mid", 0.28*misc_fact],
    ['leadlag2_mr_d', 0.47*misc_fact],
    ['pair_mr_1y', 1.4*misc_fact],
    ["hc_rb_diff_20", 0.1*misc_fact],
    ['beta_spd_trend_3m', 0.5*misc_fact],
    ['beta_spd_trend_1y', 0.5*misc_fact],
]

ferrous_strats = [
    ['io_inv_rmv_ratio_1y', 1.71*ferr_fact],
    ['io_removal_lvl', 1.04*ferr_fact],
    ['io_removal_lyoy', 0.1*ferr_fact],
    ['io_millinv_lyoy', 0.74*ferr_fact],
    ['strip_hsec_lvl_mid', 0.05*ferr_fact],
    ['steel_margin_lvl_fast', 0.195*ferr_fact],
    ["steel_sinv_lyoy_zs", 0.2*ferr_fact],
    ["steel_sinv_lyoy_mds", 0.1*ferr_fact],
    ['fef_c1_c2_ratio_or_qtl', 0.07*ferr_fact],    
    ['fef_fly_ratio_or_qtl', 0.155*ferr_fact],
    ['fef_basmom_or_qtl', 0*ferr_fact],
    ['fef_basmom5_or_qtl', 0.39*ferr_fact],
    ['fef_c1_c2_ratio_spd_qtl', 0.25*ferr_fact],    
    ['fef_basmom5_spd_qtl', 0.51*ferr_fact],
    
    #['io_pinv31_lvl_zsa', 0*ferr_fact],
    ['io_pinv45_lvl_hlr', 0.217*ferr_fact],
    ['ioarb_px_hlr', 0.06*ferr_fact],
    #['ioarb_px_hlrhys', 0.03*ferr_fact],
    
    ['ioarb_spd_qtl_1y', 0.62*ferr_fact],
    ['const_etf_mom_zsa', 0*ferr_fact],
    ['const_etf_mom_ewm', 0.164*ferr_fact],
    ['prop_etf_mom_dbth_zs', 0.178*ferr_fact],
    ['prop_etf_mom_dbth_qtl', 0.076*ferr_fact],
    ["glass_etf_mom_dbth_zs", 0.178*ferr_fact],
    ['rbsales_lyoy_mom_lt', 0.2*ferr_fact],    
    ['rb_sales_inv_ratio_lyoy', 0.2*ferr_fact],
    ['prop2hand_px_zs', 0.2*ferr_fact], 
    ['coal_mom_st_hlr', 0.15*ferr_fact],
    ['coal_mom_yr_hlr', 0.15*ferr_fact],
    
    ['steel_margin_lvl_slow', 0.4*ferr_fact],    
    ['smsf_prodcost_mom', 0.128*ferr_fact],
    ['smsf_prodcost_mom_xdemean', 0.192*ferr_fact],    
    ['mn_mine_mom', 0.03*ferr_fact],    
    ['smsf_workrate_ratio', 0.4*ferr_fact],
    ['smsf_dmd_ratio', 0.4*ferr_fact],
    ['smsf_margin_diff', 0.232*ferr_fact],
    ['smsf_coal_mom_st', 0.15*ferr_fact],
    ['smsf_coal_mom_yr', 0.12*ferr_fact], 
]

ferrous_spd_strats = [
    ['rbhc_px_diff_mds', 0.04*ferspd_fact],
    ['rbhc_px_diff_lyoy_mds', 0.0425*ferspd_fact],    
    ['rbhc_phycarry_diff_zs', 0.092*ferspd_fact],
    ['rbhc_basmom_diff_hlr', 0.217*ferspd_fact],
    ['rbhc_steel_spd_mds', 0.197*ferspd_fact],
    ['rbhc_steel_spd_lyoy_mds', 0.139*ferspd_fact],    
    ['rbhc_dmd_ratio_mds', 0.066*ferspd_fact],
    ['rbhc_dmd_ratio_lyoy_mds', 0.139*ferspd_fact],
    ['rbhc_sinv_chg_mds', 0.08*ferspd_fact],
    ['rbhc_sinv_chg_lyoy_mds', 0.053*ferspd_fact],
    ['rbhc_sinv_lratio_mds', 0.073*ferspd_fact],
    ['rbhc_sinv_lratio_lyoy_mds', 0.062*ferspd_fact],    
    ['rbhc_rbsales_lyoy_zs', 0.4*ferspd_fact],     
]

base_strats = [
    ['lme_base_ts_mds', 0.29*base_fact],
    ['lme_base_ts_mds_xdemean', 0.24*base_fact],
    ['lme_base_ts_hlr', 0*base_fact],
    ['lme_base_ts_hlr_xdemean', 0.035*base_fact],
    ['lme_futbasis_ma', 0.016*base_fact],
    ['lme_futbasis_ma_xdemean', 0.4*base_fact],     
    ['base_phybas_carry_ma', 0.354*base_fact],
    ['base_phybas_carry_ma_xdemean', 0*base_fact],   
    ['base_phybasmom_1m_zs', 0.05*base_fact],
    ['base_phybasmom_1m_zs_xdemean', 0.47*base_fact],         
    ['base_phybasmom_1y_zs', 0.028*base_fact],
    ['base_phybasmom_1y_zs_xdemean', 0*base_fact],          
    ['base_cifprem_1y_zs', 0.42*base_fact],
    ['base_cifprem_1y_zs_xdemean', 0.006*base_fact],       
    ['base_tc_1y_zs', 0.53*base_fact],
    ['base_tc_2y_zs', 0.215*base_fact],     
    ['base_inv_mds', 0.19*base_fact],
    ['base_inv_mds_xdemean', 0.31*base_fact],         
    ['base_inv_exch_ma', 0.006*base_fact],
    ['base_inv_exch_ma_xdemean', 0.164*base_fact],     
#     ['base_inv_shfe_ma', 1.0],
#     ['base_inv_shfe_ma_xdemean', 1.0],      
    ['cu_blister_rc_s_zs', 0.25*base_fact],
    ['base_etf_mom_zsa', 0.2*base_fact],
    ['base_etf_mom_ewm', 0.0*base_fact],           
    ['al_alumina_qtl', 0.038*base_fact],
    ['al_alumina_yoy_qtl', 0.106*base_fact],     
    ['al_coal_qtl', 0.176*base_fact],
    ['ni_nis_mom_qtl', 0.13*base_fact],      
    ['ni_ore_qtl', 0.14*base_fact],
    ['sn_conc_spot_hlr', 0.22*base_fact],      
]

energy_strats = [
    ["us_oil_prod_etf_mom", 1.0*energy_fact],
]

macro_strats = [
    ["shibor1m_qtl", 0.2*macro_fact],
    ['r007_lt_zs', 0.04*macro_fact],
    ['vix_zs_st', 0.2*macro_fact],
    ['cnh_cny_zs', 0.17*macro_fact],
    ['cny_dev_zs', 0.19*macro_fact],    
    ['fxbasket_zs', 0.16*macro_fact],
    ["MCU3_zs", 0.055*macro_fact],
    ["cgb_1_2_spd_zs", 0.22*macro_fact],
    ["cgb_2_5_spd_zs", 0.18*macro_fact],    
]

macro2_strats = [
    ["m1_m2_spd_zs_sgn", 0.5*macro2_fact],
    ["pmi_stl_o2inv_zs", 0.35*macro2_fact],
    ["pmi_stl_o2inv_qtl_sgn", 0.5*macro2_fact],
    #["pmi_order_rminv_ratio_zs", 0.5*macro2_fact],
]

seazn_strats = [
    ["seasonal_custom_1", 1*seazn_fact],
    ["beta_spd_seazn", 0.3*seazn_fact],
    ["long_hol_2b", 0.2*seazn_fact],    
    ["seazn_cal_mth_sr", 0.15*seazn_fact],
    ["seazn_lunar_wk2_sr", 0.25*seazn_fact],    
]


bond_strats = [
    ['bond_mr_st_qtl', 3.85*bond_fact],
    ['bond_tf_lt_qtl', 0*bond_fact],
    ['bond_carry_ma', 0.08*bond_fact],
    ["bond_tf_st_eds", 0.22*bond_fact],
    ["bond_fxbasket_zs", 0.155*bond_fact],
]

auspd_strats = [
    ["auag_cme_wratio_zs", 0.31*auspd_fact], 
    ['auag_vix_zsa_mt', 1.56*auspd_fact],
    ['auag_fxbasket_zs_yr', 0.12*auspd_fact],    
    ['auag_etf_mrev_xdemean', 0.33*auspd_fact],
    ['auag_csi500_zs_st', 0.21*auspd_fact],
]


# strat_group = prem_strats + metal_strats + misc_strats + ferrous_strats + ferrous_spd_strats + base_strats + energy_strats \
#             + macro_strats + macro2_strats + seazn_strats + bond_strats + auspd_strats
strat_group = ferrous_strats

# run backtest for a portfolio group

In [ ]:
bt_dict = {}
pnl_dict = {}
holding_dict = {}
signal_dict = {}

In [ ]:
file_folder = "C:\\dev\\data\\data_cache\\"

start_date = datetime.date(2010,1,1)
end_date = tday

portfolio_map = {
    'prod': strat_group,
}

risk_scaling = 1
asset_scaling = False
std_win = 20
trd_cost = 2e-4

pnl_tenors = ['3m', '6m', '1y', '2y', '3y', '4y', '5y', '7y', '9y', '10y', '12y', '15y']

asset_list = [
    'rb', 'hc', 'i', 'j', 'jm', 'FG', 'v', 'SM', 'SF', 'SA',
    'cu', 'al', 'zn', 'ni', 'pb', 'sn', 'ss', 'ao', 'au', 'ag', 'si', 
    'ru', 'l', 'pp', 'TA', 'MA', 'sc', 'eb', 'eg', 'UR', 'lu', 'bu', 'fu',
    'm', 'RM', 'y', 'p', 'OI', 'a', 'c', 'CF', 'jd', 'AP', 'lh', 'cs', #'bc'
    'si', 'lc', 'PK', 'CJ', 'PX', 'ec',
    'T', 'TF', 'TL',
]

trd_cost_dict = {}
for asset in asset_list:
    if asset in ['T', 'TF', 'TS', 'TL']:
        trd_cost_dict[asset] = 0.6e-4
    else:
        trd_cost_dict[asset] = trd_cost
        
product_list = [asset+'c1' for asset in asset_list]
close_prices = df.loc[:, df.columns.get_level_values(0).isin(product_list) &
                      (df.columns.get_level_values(1)=='close')].droplevel([1], axis=1).dropna(how='all').ffill()
close_prices.columns = [col[:-2] for col in close_prices.columns]
close_prices = close_prices[asset_list]
vol_df = close_prices.pct_change().rolling(std_win).std()

portfolio_name = 'prod'
run_key = portfolio_name
strat_portfolio = portfolio_map[portfolio_name]

pos_sum = pd.DataFrame(0, index=df.index, columns=asset_list)

bt_dict[run_key] = {}
signal_dict[run_key] = {}
pnl_dict[run_key] = {}
holding_dict[run_key] = {}

agg_holding = pd.DataFrame(index=df.index, columns=asset_list)
agg_signal = pd.DataFrame(index=df.index, columns=asset_list)
agg_pnl = pd.DataFrame(index=df.index, columns=asset_list)
pnl_by_signal = pd.DataFrame(index=df.index, columns=[signal_name for signal_name, weight in strat_portfolio])
port_start = pd.to_datetime('2010-01-01')

for signal_name, weight in strat_portfolio:    
    print(signal_name)
    cont_postfix = 'c1'
    if signal_name in signal_by_func:
        func = signal_by_func[signal_name]['func']
        func_args = signal_by_func[signal_name]['args']     
        signal_df = func(df, spot_df, **func_args)
    elif signal_name in signal_by_spread:
        signal_ts = get_funda_signal_from_store(spot_df, signal_name, price_df=df, signal_repo=feature_setup)
        signal_df = pd.DataFrame({asset: signal_ts * w for asset, w in signal_by_spread[signal_name]}) 
    elif signal_name in signal_by_spread2:
        signal_ts = get_funda_signal_from_store(spot_df, signal_name, price_df=df, signal_repo=feature_setup)
        spd_name = signal_by_spread2[signal_name][0]   
        nb = spread_config[spd_name][3]
        signal_df = pd.DataFrame({asset: signal_ts * w for asset, w in spread_config[spd_name][0]})
        cont_postfix = f'd{nb}'
    elif signal_name in signal_by_beta_neutral:        
        signal_ts = get_funda_signal_from_store(spot_df, signal_name, price_df=df, signal_repo=feature_setup)
        signal_df = pd.DataFrame(index=signal_ts.index)        
        for trade_asset, index_asset, w in signal_by_beta_neutral[signal_name]:
            key = f"{trade_asset}_{index_asset}"
            if trade_asset not in signal_df.columns:
                signal_df[trade_asset] = 0
            if index_asset not in signal_df.columns:
                signal_df[index_asset] = 0
            sig_df = pd.concat([
                signal_ts.to_frame('signal'),
                beta_dict[key]['trade_leg'].to_frame('trade_w'), 
                beta_dict[key]['index_leg'].to_frame('index_w')], axis=1).ffill()            
            sig_df[trade_asset] = w * sig_df['signal'] * sig_df['trade_w']
            sig_df[index_asset] = w * sig_df['signal'] * sig_df['index_w']
            signal_df[[trade_asset, index_asset]] += sig_df[[trade_asset, index_asset]]
    elif signal_name in signal_by_data:        
        signal_ts = get_funda_signal_from_store(spot_df, signal_name, price_df=df, signal_repo=feature_setup)
        signal_df = pd.DataFrame({asset: signal_ts for asset in signal_by_data[signal_name] if asset in asset_list})
    else:
        xs_type = signal_name.split("_")[-1]
        if xs_type in ['xdemean', 'xscore', 'xrank',]:
            sig_name = "_".join(signal_name.split("_")[:-1])
        else:
            sig_name = signal_name
            xs_type = 'ts'
        if sig_name in signal_by_asset:
            trade_asset = [asset for asset in asset_list if asset in signal_by_asset[sig_name]]
            signal_df = pd.DataFrame(index=df.index, columns=trade_asset)
            for asset in trade_asset:
                signal_df[asset] = get_funda_signal_from_store(spot_df, signal_name, price_df=df, asset=asset, signal_repo=feature_setup)
            if xs_type == 'xdemean':
                signal_df = xs_demean(signal_df)
            elif xs_type == 'xscore':
                signal_df = xs_score(signal_df)
        else:
            print("unsupported signal")
    
    if cont_postfix == 'c1':
        signal_exec_param = signal_execution_config.get(signal_name, {"win": "n305", "lag": 1})
        cost_dict = trd_cost_dict
    else:
        signal_exec_param = {"win": "close", "lag": 1}
        cost_dict = {}
    
    traded_products = [f'{col}{cont_postfix}' for col in signal_df.columns]    
    if signal_name in signal_store:
        _, _, _, _, _, _, _, post_func, _, _ = signal_store[signal_name][1]
        last_func = post_func.split('|')[-1]
        if 'buf' in last_func:
            buffer_size = float(last_func[3:])
            signal_df = signal_buffer(signal_df, buffer_size)
        elif 'bfc' in last_func:
            buffer_size = float(last_func[3:])
            signal_df = signal_cost_optim(signal_df, buffer_size, vol_df[signal_df.columns],
                                          cost_dict=trd_cost_dict,
                                          turnover_dict={},
                                          power=3)        
        
    signal_df = signal_df.reindex(index=cdates).ffill().reindex(index=bdates).fillna(0)
    if signal_name in signal_by_spread2:
        spd_name = signal_by_spread2[signal_name][0]
        nb = spread_config[spd_name][3]
        vwin = spread_config[spd_name][1]
        spd_px = pd.DataFrame({asset: df[(f"{asset}d{nb}", "close")] * w for asset, w in spread_config[spd_name][0]}).sum(axis=1)
        vol_ts = spd_px.diff().dropna().rolling(vwin).std()
        vol_factor = pd.DataFrame({asset: vol_ts for asset, w in spread_config[spd_name][0]}).reindex(index=df.index).ffill()
    else:
        vol_factor = vol_df[signal_df.columns]
    traded_prices = df.loc[:, df.columns.get_level_values(0).isin(traded_products) &
                      (df.columns.get_level_values(1)==signal_exec_param['win'])].droplevel([1], axis=1).dropna(how='all').ffill()
    traded_prices.columns = [col[:-2] for col in traded_prices.columns]
    traded_prices = traded_prices[signal_df.columns]
    if cont_postfix in ['d1', 'd2']:
        df_pxchg = traded_prices.diff()        
    else:
        df_pxchg = traded_prices.pct_change()
    holding = generate_holding_from_signal(signal_df, vol_factor, 
                                           risk_scaling=risk_scaling, 
                                           asset_scaling=asset_scaling)    
    bt_metrics = MetricsBase(holdings=holding[signal_df.columns],
                             returns=df_pxchg[signal_df.columns], 
                             shift_holdings=signal_exec_param["lag"],
                             cost_dict=cost_dict
                            )
    bt_dict[run_key][signal_name] = bt_metrics
    signal_dict[run_key][signal_name] = signal_df
    holding_dict[run_key][signal_name] = holding

    pnl_stats = bt_metrics.calculate_pnl_stats(shift=0, tenors=pnl_tenors, perf_metrics=['sharpe', 'std', 'sortino', 'calmar'])
    perf_stats = transform_output(pnl_stats)
    print(f"\nsignal={signal_name}\n")
    display(perf_stats.round(2))    
    
    if signal_name in signal_by_spread2:    
        asset_pnl = pnl_stats['asset_pnl']
    else:
        asset_pnl = bt_metrics.calculate_daily_pnl(traded_prices, close_prices[signal_df.columns], mode='ret')
    
    pnl_dict[run_key][signal_name] = asset_pnl
    
    iplot(asset_pnl.cumsum().sum(axis=1).to_frame("total")[port_start:], title=signal_name)
    iplot(asset_pnl.cumsum()[port_start:], title=signal_name)    
    
    display(pd.concat([pnl_stats['pnl_per_trade'].to_frame("bias"), pnl_stats['turnover'].to_frame("TO")], axis=1).round(2))
    display(pnl_stats['asset_sharpe_stats'].round(2))
    
    agg_pnl = agg_pnl.add((asset_pnl*weight).reindex_like(agg_pnl), fill_value=0)
    pnl_by_signal[signal_name] = (asset_pnl*weight).sum(axis=1)
    if signal_name in signal_by_spread2:        
        adj_signal_df = signal_df.mul(close_prices[signal_df.columns].reindex_like(agg_signal).ffill())
        agg_signal = agg_signal.add(adj_signal_df.reindex_like(agg_signal).ffill(), fill_value=0)
        adj_holding_df = holding.mul(close_prices.shift(1)[signal_df.columns].reindex_like(agg_holding).ffill())
        agg_holding = agg_holding.add(adj_holding_df.reindex_like(agg_holding).ffill(), fill_value=0)
    else:
        agg_signal = agg_signal.add((signal_dict[run_key][signal_name]*weight).reindex_like(agg_signal).ffill(), fill_value=0)
        agg_holding = agg_holding.add((holding_dict[run_key][signal_name]*weight).reindex_like(agg_holding).ffill(), fill_value=0)

met_list = []
for asset in agg_pnl:
    sr_ts = calc_perf_by_tenors(agg_pnl[asset], tenors=pnl_tenors, metric='sharpe')
    met_list.append(sr_ts.to_frame(asset))
    
sr_df = pd.concat(met_list, axis=1)

port_pnl = agg_pnl.sum(axis=1)
met_list = []
for metric in ['sharpe', 'std']:
    sr_ts = calc_perf_by_tenors(port_pnl, tenors=pnl_tenors, metric=metric)
    sr_ts.index=pnl_tenors
    met_list.append(sr_ts.to_frame(metric))
port_df = pd.concat(met_list, axis=1)

display(port_df.round(3))
display(sr_df.round(3))    

iplot(agg_pnl.sum(axis=1).cumsum().to_frame('portfolio pnl'))
iplot(agg_pnl.cumsum(), title="portfolio pnl by asset")

# print(port_pnl[-40:])
# port_pnl.to_csv(file_folder + "port_pnl.csv")


In [ ]:
print("\nsignal:\n")
display(agg_signal.tail(20))

# print("\nholding:\n")
# display(agg_holding.tail(20))

In [ ]:
display(port_pnl['2021':].cumsum()[-40:].astype('int'))

In [ ]:
prem_strat_names = [elem[0] for elem in prem_strats]
metal_strat_names = [elem[0] for elem in metal_strats]
misc_strat_names = [elem[0] for elem in misc_strats]
ferrous_strat_names = [elem[0] for elem in ferrous_strats]
ferrous_spd_strat_names = [elem[0] for elem in ferrous_spd_strats]
base_strat_names = [elem[0] for elem in base_strats]
energy_strat_names = [elem[0] for elem in energy_strats]
macro_strat_names = [elem[0] for elem in macro_strats]
macro2_strat_names = [elem[0] for elem in macro2_strats]
auspd_strat_names = [elem[0] for elem in auspd_strats]
seazn_strat_names = [elem[0] for elem in seazn_strats]
bond_strat_names = [elem[0] for elem in bond_strats]
auspd_strat_names = [elem[0] for elem in auspd_strats]

In [ ]:
data_dict = {}
data_dict['prem_pnl'] = pnl_by_signal[prem_strat_names].sum(axis=1)
data_dict['misc_pnl'] = pnl_by_signal[misc_strat_names].sum(axis=1)
data_dict['metal_pnl'] = pnl_by_signal[metal_strat_names].sum(axis=1)
data_dict['ferrous_pnl'] = pnl_by_signal[ferrous_strat_names].sum(axis=1)
data_dict['ferrous_spd_pnl'] = pnl_by_signal[ferrous_spd_strat_names].sum(axis=1)
data_dict['base_pnl'] = pnl_by_signal[base_strat_names].sum(axis=1)
data_dict['energy_pnl'] = pnl_by_signal[energy_strat_names].sum(axis=1)
data_dict['macro_pnl'] = pnl_by_signal[macro_strat_names].sum(axis=1)
data_dict['macro2_pnl'] = pnl_by_signal[macro2_strat_names].sum(axis=1)
data_dict['seazn_pnl'] = pnl_by_signal[seazn_strat_names].sum(axis=1)
data_dict['bond_pnl'] = pnl_by_signal[bond_strat_names].sum(axis=1)
data_dict['auspd_pnl'] = pnl_by_signal[auspd_strat_names].sum(axis=1)
pnl_df = pd.DataFrame(data_dict)

corr_cutoff = '2019-01-01'
print("std:\n%s\n" % pnl_df[corr_cutoff:].std(axis=0))
print("std (total) = %s" % pnl_df[corr_cutoff:].sum(axis=1).std())
iplot(pnl_df[corr_cutoff:].cumsum())

In [ ]:
sr_list = []
for strat in pnl_df.columns:
    sr = calc_perf_by_tenors(pnl_df[strat],tenors=pnl_tenors, metric='sharpe')
    sr_list.append(sr.to_frame(strat))
sr_df = pd.concat(sr_list, axis=1)

display(sr_df)



In [ ]:
display(pnl_df[corr_cutoff:].corr())